In [ ]:
from ultralytics import YOLO
import supervision as sv
import numpy as np
import pandas as pd


def run_bytetrack(weight_path, source_path, target_path)

    model = YOLO(weight_path)

    tracker = sv.ByteTrack(minimum_matching_threshold=0.8, track_activation_threshold=0.6, lost_track_buffer=40)

    bounding_box_annotator = sv.BoundingBoxAnnotator()
    label_annotator = sv.LabelAnnotator()

    def callback(frame: np.ndarray, index: int) -> np.ndarray:
        results = model(frame)[0]
        detections = sv.Detections.from_ultralytics(results)
        detections = tracker.update_with_detections(detections)

        labels = [f"#{tracker_id}" for tracker_id in detections.tracker_id]

        annotated_frame = bounding_box_annotator.annotate(
            scene=frame.copy(), detections=detections)
        annotated_frame = label_annotator.annotate(
            scene=annotated_frame, detections=detections, labels=labels)
        return annotated_frame

    sv.process_video(
        
        source_path=source_path,
        target_path=target_path,
        callback=callback
    )

In [ ]:


class ByteTrack:

    def ret_frame_dets(self, tensor, i, row_number, type):
        """
        Retrieves detections for a specific frame and row.
        
        Parameters:
            tensor (list): List of detection results per frame.
            i (int): Frame index.
            row_number (int): Row index (0-7).
            type (int): Video type (1-4).
        
        Returns:
            DataFrame: DataFrame containing detections for the specified row.
        """
        xyxy = tensor[i].boxes.xyxy
        conf = tensor[i].boxes.conf
        cxywh = tensor[i].boxes.xywh  # Center x, center y, w, h of bbox

        df1 = pd.DataFrame(xyxy, columns=['xmin', 'ymin', 'xmax', 'ymax'])
        df2 = pd.DataFrame(conf, columns=['conf'])
        df3 = pd.DataFrame(cxywh, columns=['CenterX', 'CenterY', 'w', 'h'])

        df_appended = pd.concat([df1, df2, df3], axis=1)
        final_df = df_appended[df_appended['ymin'] >= 660]  # Detections with ymin >= 660

        rows = self.return_rowlist(final_df, type)
        temp_df = rows[row_number]  # Rows ranging from 0-7 (1-8)
        temp_df = temp_df.drop(['CenterX', 'CenterY', 'w', 'h'], axis=1)
        return temp_df
    
    def get_bt_count(self, window_listup):
        """
        Counts unique tracked objects using ByteTrack.
        
        Parameters:
            window_listup (list): List of detections per frame.
        
        Returns:
            int: Count of unique tracked objects.
        """
        tracker = sv.ByteTrack(minimum_matching_threshold=0.8,
                               track_activation_threshold=0.6,
                               lost_track_buffer=40)
        id_list = []
        track_ids = []

        for i in range(60):
            results = window_listup[i]
            boxes, confidences = results
            detections = sv.Detections(
                xyxy=boxes,
                confidence=confidences,
                class_id=np.zeros(len(confidences), dtype=int)
            )
            detections = tracker.update_with_detections(detections)
            ids = detections.tracker_id.tolist()
            track_ids.append(ids)

        for frame_ids in track_ids:
            id_list.extend(frame_ids)

        count = len(set(id_list))
        return count
    
    # for j in range(60):
    #                 df = self.ret_frame_dets(model_resultz, j, row_number[i], type)
    #                 frame_detections = self.ret_trimmed_df(df, ymin, ymax)
    #                 results_window.append(frame_detections)

    #             crops_count = self.get_bt_count(results_window)
    #             count_in_row.append(crops_count)